# Limpieza y preprocesamiento de datos

En este notebook se realiza la carga, inspección, limpieza y preparación del dataset
original de enfermedades y síntomas, con el objetivo de generar un conjunto de datos
final apto para el entrenamiento de modelos de aprendizaje automático.


## Importación de librerías

In [3]:
import pandas as pd
import numpy as np
import json


## Carga del dataset original

In [4]:
ruta = "../datos/originales/Final_Augmented_dataset_Diseases_and_Symptoms.csv"
df = pd.read_csv(ruta)

df.head()


,diseases,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,panic disorder,1,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,panic disorder,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,panic disorder,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,panic disorder,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,panic disorder,1,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## Inspección inicial de los datos

### Dimensiones del dataset

In [5]:
df.shape

(246945, 378)

El dataset original contiene **246.945 registros** y **378 variables**. Cada registro
representa un caso simulado de paciente, donde una variable corresponde a la enfermedad
diagnosticada y el resto a la presencia o ausencia de síntomas.


### Información general

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246945 entries, 0 to 246944
Columns: 378 entries, diseases to neck weakness
dtypes: int64(377), object(1)
memory usage: 712.2+ MB


La inspección de tipos de datos muestra que la variable objetivo `diseases` es de tipo
categórico, mientras que el resto de variables son numéricas enteras, correspondientes a
síntomas codificados de forma binaria. No se detectan valores faltantes en el dataset.


### Valores nulos

In [7]:
df.isnull().sum().sort_values(ascending=False).head(10)

neck weakness                       0
diseases                            0
anxiety and nervousness             0
depression                          0
shortness of breath                 0
depressive or psychotic symptoms    0
sharp chest pain                    0
dizziness                           0
insomnia                            0
abnormal involuntary movements      0
dtype: int64

El análisis de valores nulos indica que no existen datos faltantes en ninguna de las
variables del dataset, lo que permite evitar técnicas de imputación y simplifica el
proceso de preprocesamiento.


## Análisis de la variable objetivo

In [8]:
df['diseases'].value_counts().head(10)

diseases
cystitis                          1219
nose disorder                     1218
vulvodynia                        1218
complex regional pain syndrome    1217
spondylosis                       1216
vaginal cyst                      1215
esophagitis                       1215
peripheral nerve disorder         1215
hypoglycemia                      1215
conjunctivitis due to allergy     1215
Name: count, dtype: int64

In [9]:
df['diseases'].nunique()

773

Se identifican **773 enfermedades distintas**, con una distribución relativamente
equilibrada en el número de registros por clase, lo que resulta adecuado para el
entrenamiento de modelos de clasificación multiclase.


## Comprobación de duplicados

In [10]:
df.duplicated().sum()

np.int64(57298)

El resultado indica que existen **57.298 filas duplicadas** en el dataset original. 
Estas filas representan registros repetidos con la misma combinación de síntomas y
enfermedad, por lo que se procede a su eliminación para evitar sesgos en el
entrenamiento de los modelos.

In [17]:
filas_antes = df.shape[0]
df = df.drop_duplicates()
filas_despues = df.shape[0]
filas_antes, filas_despues


(189647, 189647)

Tras la eliminación de duplicados, el número de registros se reduce de **246.945** a
**189.647 filas**, manteniendo el mismo número de variables. Este proceso mejora la
calidad del conjunto de datos sin pérdida de información relevante.


## Limpieza y normalización básica

### Normalización de nombres de columnas

In [12]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

La normalización de los nombres de las columnas facilita su manejo posterior en código,
evitando problemas derivados de espacios, mayúsculas o caracteres inconsistentes.


### Verificación de valores binarios

In [13]:
df.drop(columns=['diseases']).isin([0,1]).all().value_counts()

True    377
Name: count, dtype: int64

La verificación confirma que todas las variables de entrada correspondientes a síntomas
están correctamente codificadas como valores binarios (0 = ausencia del síntoma,
1 = presencia del síntoma), lo que permite su uso directo en modelos de clasificación
supervisada sin necesidad de transformaciones adicionales.

## Dataset final

Tras el proceso de limpieza y verificación, se obtiene un dataset final sin valores
nulos ni duplicados, listo para su uso en el entrenamiento de modelos de aprendizaje
automático.

## Guardado del dataset procesado

In [15]:
df.to_csv(
    "../datos/procesados/enfermedades_sintomas_limpio.csv",
    index=False
)

El dataset procesado se guarda en formato CSV y constituye la versión definitiva que será
utilizada en todas las fases posteriores del proyecto, incluyendo el análisis
exploratorio, el entrenamiento y la evaluación de modelos.


## Creación del mapeo de características

In [16]:
sintomas = list(df.columns)
sintomas.remove("diseases")

with open("../datos/procesados/mapeo_caracteristicas.json", "w") as f:
    json.dump(sintomas, f, indent=4)


El archivo JSON generado contiene la lista de síntomas utilizados como variables de
entrada del modelo. Este mapeo será empleado en el prototipo para transformar la
selección de síntomas del usuario en un vector compatible con el modelo entrenado.


## Conclusiones del preprocesamiento

El proceso de limpieza ha permitido obtener un conjunto de datos estructurado,
coherente y apto para el entrenamiento de modelos de clasificación supervisada.
Este dataset será utilizado en las siguientes fases del proyecto para el análisis
exploratorio y el desarrollo de modelos predictivos.


In [18]:
df.shape

(189647, 378)

El dataset final cuenta con **189.647 registros** y **378 variables**, de las cuales una
corresponde a la variable objetivo (enfermedad) y el resto a síntomas binarios. Este
conjunto de datos constituye la base definitiva para el análisis exploratorio y el
entrenamiento de los modelos predictivos del proyecto.

## Uso de datasets complementarios

Este enfoque permite separar claramente el proceso de entrenamiento del modelo de la
interacción con el usuario final, garantizando un diseño modular y una mejor
interpretabilidad del sistema desarrollado.